# Resource Estimation

At MIT, in 1977, [Ron **R**ivest](https://en.wikipedia.org/wiki/Ron_Rivest), [Adi **S**hamir](https://en.wikipedia.org/wiki/Adi_Shamir), and [Leonard **A**dleman](https://en.wikipedia.org/wiki/Leonard_Adleman) introduced the [RSA](https://en.wikipedia.org/wiki/RSA_(cryptosystem)) cryptosystem. Today, their encryption algorithm sees wide-ranging applications; securing emails, network traffic in VPNs, and even your bank account transactions.

Given its uses in such sensitive domains, one would think that RSA is unbreakable; perhaps its inner workings are kept secret so no one can find out its vulnerabilites, or maybe it's just too complex for anyone to understand. Surprisingly, RSA's implementation is neither secret nor complex, and the algorithm that breaks it has been known since the day it was introduced.

In fact, we can write that algorithm ourselves. At the heart of RSA is a known number, $r$, that has two unknown prime factors, $p$ and $q$. All we need to decrypt an RSA-encrypted ciphertext is to find $p$ and $q$.

In [6]:
def factorize(r: int) -> tuple[int, int]:
	"""
	Factorizes the input into its two prime factors.

	Parameters
	----------
	r
		The number to factorize.
		Must be a positive composite integer that has exactly two prime factors.

	Returns
	-------
	tuple[int, int]
		The two prime factors of the input number.

	Raises
	------
	ValueError
		If the input is prime.
	"""
	from math import floor, sqrt

	# We handle the special case of 2 separately so we can optimize later
	if r % 2 == 0:
		p = 2
		q = r // 2
		return (p, q)

	# 𝑝 > 2 because we already checked if 𝑟 was an even number
	lower_limit = 3

	# 𝑝 < ⌊√𝑟 + 1⌋ because 𝑟 cannot have factors larger than √𝑟
	upper_limit = floor(sqrt(r) + 1)

	# 𝑝 is prime ⇒ 𝑝 is not an even number
	# ∴ we can take steps of size 2, if we start from 3
	step_size = 2

	for p in range(lower_limit, upper_limit, step_size):
		if r % p == 0:
			q = r // p
			return (p, q)

	# Exiting loop without finding a factor ⇒ 𝑟 is prime
	raise ValueError("r is prime")

To test our code, let's assume $p = 43$ and $q = 97$. Thus, $r = pq = 43 \cdot 97 = 4171$.

Additionally, we will time our code to see how long it takes to find $p$ and $q$.

In [11]:
from lib import time

r = 4171
time_taken = time(lambda: factorize(r))
(p, q) = factorize(r)

print("Factorized r in", time_taken, "s")
print("(p, q) =", (p, q))

Factorized r in 1.92900188267231e-06 s
(p, q) = (43, 97)


On most modern computers, factorizing $4171$ only takes a few microseconds. That sounds like bad news for RSA, but we should try a few more numbers before coming to any conclusions.

We will now take increasingly larger numbers and factorize them to see how long it takes.

Resource estimation is a crucial aspect of both classical and quantum computing. It involves predicting the computational resources required to execute a given algorithm. These resources can include time, memory, and other hardware-specific metrics.

In this tutorial, we will explore resource estimation by comparing the performance of arrays and linked lists in a classical setting.

## Arrays

An array is a data structure that stores a fixed-size collection of elements of the same type. The elements are stored in contiguous memory locations, which allows for fast access to individual elements. However, inserting or deleting elements in the middle of an array can be slow, as it requires shifting all subsequent elements.

![Array](./assets/array.png)

## Linked Lists

A linked list is a data structure that consists of nodes, where each node contains a value and a reference to the next node in the list. This structure allows for efficient insertion and deletion of elements, as it does not require shifting elements in memory. However, accessing individual elements in a linked list can be slower than in an array, as it requires traversing the list from the beginning.

![List](./assets/list.png)

### A Counterintuitive Comparison

Consider the following algorithm:
1. Generate $n$ random integers.
2. Insert them into a sequence in numerical order.  
For example, if our random integers were $5$, $1$, $4$, and $2$, the sequence would grow as follows:  
	* $5$
	* $1$, $5$
	* $1$, $4$, $5$
	* $1$, $2$, $4$, $5$
3. Remove elements from the sequence, one at a time, choosing the element to remove at random.  
For example, if we chose to remove the elements in the order $2$, $5$, $1$, and $4$, the sequence would shrink as follows:  
	* $1$, $2$, $4$, $5$
	* $1$, $4$, $5$
	* $1$, $4$
	* $4$
4. Halt when the sequence is empty.

Since the algorithm involves lots of insertions and deletions, we might expect a linked list to outperform an array. However, the situation is more nuanced than it seems. In this tutorial, we will analyze the resource requirements of this algorithm for both arrays and linked lists.

In [4]:
# Ask the students for their implementation; might use it to benchmark against the solution

def student_solution():
	pass

#### Time Complexity

We will now attempt to estimate the time complexity of the algorithm for both arrays and linked lists.

Step 1 involves generating $n$ random integers, which can be done in $O(n)$ time for both arrays and linked lists. Therefore, we do not need to include this step in our analysis.

Steps 2 and 3 involve first finding the correct position before insertion / deletion can take place. To keep things fair, we will make both arrays and linked lists perform linear search to find the correct position. This operation will take $O(n)$ time for both arrays and linked lists.

That leaves only the insertion and deletion steps to consider. Let's analyze the time complexity of these steps for both arrays and linked lists.

#### Array Implementation Complexity

To calculate the complexity of the algorithm when using an array, we need to consider the time required for inserting and deleting elements. Inserting an element into an array of size $n$ requires $O(n)$ time, as we may need to shift all subsequent elements. Deleting an element from an array also requires $O(n)$ time, as we may need to shift all subsequent elements. Therefore, the total time complexity of the algorithm when using an array is $O(n^2)$.

#### Linked List Implementation Complexity

To calculate the complexity of the algorithm when using a linked list, we need to consider the time required for inserting and deleting elements. Inserting an element into a linked list of size $n$ requires $O(n)$ time, as we may need to traverse the list to find the correct position. Deleting an element from a linked list also requires $O(n)$ time, as we may need to traverse the list to find the element to delete. Therefore, the total time complexity of the algorithm when using a linked list is $O(n^2)$.

 In theory, both arrays and linked lists have the same time complexity for this algorithm. However, in practice, arrays may outperform linked lists due to cache locality and other hardware-specific optimizations. In this tutorial, we will compare the performance of arrays and linked lists for this algorithm using Python.

In [5]:
# Benchmarks and plots of the array v linked list solution
# Help the students see the different O(n^2) curves.

### Bjarne Stroustrup C++ Results

[Video link](https://youtu.be/YQs6IC-vgmo?si=0U3VzesKsXyhNPCT)

![C++](./assets/vector-list-perf.png)

Finally, we show the shocking result from C++, where arrays always beat linked lists. This is due to the fact that the C++ standard library implementation of `std::vector` is highly optimized on modern CPU hardware, effectively utilizing caches and memory prefetching to outperform linked lists.